In [1]:
# install dependencies
!pip install transformers
!pip install einops
!pip install accelerate

# use these for running remote inference at huggingface
!pip install huggingface_hub
!pip install langchain
!pip install langchain-community
!pip install langchain-core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.0 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36


In [2]:
import os
from getpass import getpass
os.environ['HUGGINGFACEHUB_API_TOKEN'] = getpass('Enter huggingfacehub api token: ')

Enter huggingfacehub api token: ··········


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

# load model
model = "tiiuae/falcon-7b-instruct"
# model = "tiiuae/falcon-40b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model)

falcon_pipeline = transformers.pipeline("text-generation",
                                        model=model,
                                        tokenizer=tokenizer,
                                        torch_dtype=torch.bfloat16,
                                        trust_remote_code=True,
                                        device_map="auto"
                                        )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/17.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

In [32]:
# load remote model
from langchain.llms import HuggingFaceHub
model = "tiiuae/falcon-7b-instruct"
# model = "tiiuae/falcon-40b-instruct"
falcon = HuggingFaceHub(
    repo_id=model,
    model_kwargs={"temperature": 0.4,
                  "max_length": 128},
)


I tried temperatures of 0.5, 0.1, and 1

In [33]:
def get_completion_falcon(input, host="remote", **kwargs):
    prompt = f"#### User: \n{input}\n\n#### Response from falcon-7b-instruct:"
    response = ""
    # print(prompt)
    if host.lower() == "local":
      print("invoking llm at Google Colab")
      if 'max_length' not in kwargs:
        kwargs['max_length'] = 1000

      falcon_response = falcon_pipeline(prompt,
                                      #max_length=500,
                                      do_sample=True,
                                      top_k=10,
                                      num_return_sequences=1,
                                      eos_token_id=tokenizer.eos_token_id,
                                      **kwargs,
                                      )
      response = falcon_response[0]['generated_text']

    elif host.lower() == "remote":
      print("invoking llm at Huggingface Hub")
      if "max_length" in kwargs:
        kwargs['max_new_tokens'] = kwargs['max_length']

      response = falcon.invoke(prompt, **kwargs)

    else:
      print ("invalid host value, must be 'remote' or 'local'")

    return response

#Few-shot chain-of-thought

In [35]:
kwargs = {"max_length": 2200}
few_shot_COT_prompt = """You are a mathematical genius.
The template of the question is given as Q: [question]. The template of the answer is given as A: [answer].
Help me solve the problem below using the examples given.

Examples:
Q: The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: The given numbers are: (4, 8, 9, 15, 12, 2, 1). Next select the odd numbers (9, 15, 1).  Adding all the selected odd numbers (9, 15, 1) gives 25. 25 is an odd number, hence the answer is False.

Q: The odd numbers in this group add up to an even number: 17, 10, 19, 4, 8, 12, 24.
A: The given numbers are: (17, 10, 19, 4, 8, 12, 24). Next select the odd numbers (17, 19).  Adding all the selected odd numbers (17, 19) gives 36. 36 is an even number, hence the answer is True.

Q: The odd numbers in this group add up to an even number: 16, 11, 14, 4, 8, 13, 24.
A: The given numbers are: (16, 11, 14, 4, 8, 13, 24). Next select the odd numbers (11, 13).  Adding all the selected odd numbers (11, 13) gives 24. 24 is an even number, hence the answer is True.

Q: The odd numbers in this group add up to an even number: 17, 9, 10, 12, 13, 4, 2.
A: The given numbers are: (17, 9, 10, 12, 13, 4, 2). Next select the odd numbers (17, 9, 13).  Adding all the selected odd numbers (17, 9, 13) gives 39. 39 is an odd number, hence the answer is False.

Problem:
Q: The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.
A:
"""
#15 + 5 + 13 + 7 + 1 = 41
few_shot_COT_prompt_explanation = get_completion_falcon(few_shot_COT_prompt, **kwargs)
print(few_shot_COT_prompt_explanation)

invoking llm at Huggingface Hub
#### User: 
You are a mathematical genius.
The template of the question is given as Q: [question]. The template of the answer is given as A: [answer].
Help me solve the problem below using the examples given.

Examples:
Q: The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: The given numbers are: (4, 8, 9, 15, 12, 2, 1). Next select the odd numbers (9, 15, 1).  Adding all the selected odd numbers (9, 15, 1) gives 25. 25 is an odd number, hence the answer is False.

Q: The odd numbers in this group add up to an even number: 17, 10, 19, 4, 8, 12, 24.
A: The given numbers are: (17, 10, 19, 4, 8, 12, 24). Next select the odd numbers (17, 19).  Adding all the selected odd numbers (17, 19) gives 36. 36 is an even number, hence the answer is True.

Q: The odd numbers in this group add up to an even number: 16, 11, 14, 4, 8, 13, 24.
A: The given numbers are: (16, 11, 14, 4, 8, 13, 24). Next select the odd numbers (11, 13).  Adding a

#Few-shot

In [26]:
kwargs= {"max_length": 900}

few_shot_prompt =  """I will provide you with a few examples of a math problem.
Follow the pattern to answer the question at the end.
Example 1:
Problem: 'The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.'
Answer: 'The answer is false.'

Example 2:
Problem: 'The odd numbers in this group add up to an even number: 17, 10, 19, 4, 8, 12, 24.'
Answer: 'The answer is true.'

Now do this for this set of numbers: 15, 32, 5, 13, 82, 7, 1. """
response_few_shot = get_completion_falcon(few_shot_prompt, **kwargs)
print(response_few_shot)

invoking llm at Huggingface Hub
#### User: 
I will provide you with a few examples of a math problem.
Follow the pattern to answer the question at the end.
Example 1:
Problem: 'The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.'
Answer: 'The answer is false.'

Example 2:
Problem: 'The odd numbers in this group add up to an even number: 17, 10, 19, 4, 8, 12, 24.'
Answer: 'The answer is true.'

Now do this for this set of numbers: 15, 32, 5, 13, 82, 7, 1. 

#### Response from falcon-7b-instruct:

The answer is:

[Falcon-7b-instruct] 'The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.' The odd numbers (1, 5, and 9) are not included in the sum of these odd numbers that add up to an even number (15), therefore, the answer is 'false'.

In Excro's set of numbers, the odd numbers (1, 5, and 9) are not included in the sum of these odd numbers that add up to an even number (13), therefore, the answer is 'true'.


#Chain-of-thought

In [27]:
math_problem_prompt = "Explain whether the following statement is true or not. The odd numbers in (15, 32, 5, 13, 82, 7, 1) add up to an even number."
#15, 32, 5, 13, 82, 7, 1
kwargs = {"max_length": 1200}
math_problem_solution = get_completion_falcon(math_problem_prompt, **kwargs)
print(math_problem_solution)

invoking llm at Huggingface Hub
#### User: 
Explain whether the following statement is true or not. The odd numbers in (15, 32, 5, 13, 82, 7, 1) add up to an even number.

#### Response from falcon-7b-instruct:

True

#### Test statistics:
The file contains 14 questions and 3 test objects that have been used in this model.
As 15 + 32 + 5 = 82, and 58 + 81 = 139 with the sum of two odd numbers being an even number, it can be concluded that the odd numbers in the query do add up to an even number.


#Explanation

The combined few-shot/chain-of-thought prompt tends to get the incorrect answer. Of the runs I did it would get the correct sum sometimes, however it would still identify the sum adding up to an even number as false. I was able to get it to reply true once, but the reasoning it used was fairly creative.

The few-shot example was a bit harder but with more concise language I was able to get the correct answer fairly quickly.

The chain-of-thought example was very difficult for this specific question. Not only did it take me several iterations to even illicit the correct answer of true, but the reasoning behind how it got that answer was still flawed. Chain-of-thought seems to work better for open-ended questions more so than questions searching for a specific answer.

A lower temperature actually skewed my results more towards chain-of-thought being more effective. A temp of 0.1 gave the correct answer here immediately. The results are also more deterministic overall and seem to work better for the math problem. A higher temperature kind of went off the rails and gave answers for "creative interpretations" of the original question. In more scientific terms, the higher temperatures seems to correlate with a higher degree of stochasticity/randomness. This is probably more useful when you are looking for a more creative answer and less suited for our math problem.